In [390]:
import pandas as pd
from urllib.parse import quote
from google.colab import files
import numpy as np
import datetime
import re
from dateutil.relativedelta import relativedelta
from tqdm import tqdm

In [386]:
#google drive에 있는 파일 사용
# from google.colab import drive
# drive.mount("/content/gdrive")


#파일 직접 업로드
from google.colab import files
uploaded = files.upload()
#1923_dramalist.csv 업로드

Saving 1923_dramalist.csv to 1923_dramalist (2).csv


In [387]:
file_path = "/content/1923_dramalist.csv"
drama_list = pd.read_csv(file_path, index_col = 0)
drama_list

,작품명,극본,연출,방송 기간,방송,방송사,유형,UHD,방송.1,시청,비고
0,"신비아파트 외전: 기억, 하리 2",이인혜,박용진,"2019.02.15, 2019.04.05",8,투니버스,금,NaN,NaN,(),1회 연장
1,너를 싫어하는 방법,"유소원, 정수현",오관진,"2019.04.1, 2019.04.2",2,"JTBC2,4","(월화), 단막",NaN,NaN,(),8회 연장
2,아름다운 세상,김지우,박찬홍,"2019.05.25, 2019.04.05",16,JTBC,금토,NaN,NaN,(),8회 연장
3,국민 여러분!,한정훈,김정현,"2019.05.28, 2019.04.01",36 (18),KBS2,월화,NaN,NaN,(),8회 연장
4,날 녹여주오,백미경,신우철,"2019.09.28, 2019.11.17",16,tvN,토일,NaN,NaN,(),사전제작
...,...,...,...,...,...,...,...,...,...,...,...
415,가슴이 뛴다,"정승주, 김하나","이민수, 이현석",2023.6.26,16,KBS2,월화,NaN,NaN,(),사전제작
416,기적의 형제,김지우,박찬홍,2023.6.28,16,JTBC,수목,NaN,NaN,(),사전제작
417,닥터 차정숙,정여랑,김대진,"2023.6.4, 2023.4.15",16,JTBC,토일,NaN,NaN,(),사전 제작
418,이로운 사기,한우주,이수현,"2023.7.18, 2023.5.29",16,tvN,월화,NaN,NaN,(),사전제작


# 시청률 데이터 추출

## functions

In [407]:
# 드라마 제목을 위키피디아에 검색해 html데이터를 읽어옴.
def read_rating(title):
    front_url = "https://ko.wikipedia.org/wiki/"
    encoded_url = quote(front_url + title, safe=':/_')
    rating_df = pd.DataFrame()

    table_matching_list = ['제1회', '제1화', '1화']

    if not title == '메모리스트': #웹툰이 먼저 걸림
        try:
            tables = pd.read_html(encoded_url)
            for table in tables:
                if table.iloc[0,0] in table_matching_list:
                    rating_df = table
                    break
        except:
            pass

    #위에서 제대로 시청률을 읽지 못한 경우->_(드라마)를 붙여서 검색해줘야 함
    if rating_df.empty:
        encoded_url = quote(front_url+title+"_(드라마)", safe=':/_')
        try:
            tables = pd.read_html(encoded_url)
            for table in tables:
                if table.iloc[0,0] in table_matching_list:
                    rating_df = table
                    break
        except:
            pass

    #위에서 제대로 시청률을 읽지 못한 경우->_(대한민국의 드라마)를 붙여서 검색해줘야 함
    if rating_df.empty:
        encoded_url = quote(front_url+title+"_(대한민국의 드라마)", safe=':/_')
        try:
            tables = pd.read_html(encoded_url)
            for table in tables:
                if table.iloc[0,0] in table_matching_list:
                    rating_df = table
                    break
        except:
            pass

    return rating_df


In [408]:
# multi index를 단일 index로 바꾸고
# 'TNmS 시청률[2]' 형태의 column name을 'tnms'로 바꿈
def edit_multi_index(multi_cols):
    cols = [tpl[1] for tpl in multi_cols[:]]
    for idx in range(len(cols)):
        if 'TNmS'.lower() in cols[idx].lower():
            cols[idx] = 'TNmS'.lower()
        elif 'AGB'.lower() in cols[idx].lower():
            cols[idx] = 'AGB'.lower()
        # 닐슨 코리아 시청률의 경우, 자료에 많이 없으므로 그냥 AGB랑 동등하게 취급
        elif '닐슨 코리아'.lower() in cols[idx].lower():
            cols[idx] = 'AGB'.lower()
    return cols


#평균 시청률 또는 평균 제외
def remove_avg_rating(rating_df):
    if ((rating_df["회차"] == '평균 시청률') | (rating_df["회차"] == '평균')).any():
        removing_idx_bool = (rating_df["회차"] == '평균 시청률') | (rating_df["회차"] == '평균')
        removing_idx = list(rating_df[removing_idx_bool].index)[0]
        rating_df = rating_df.iloc[:removing_idx, :].copy()
    else:
        pass
    return rating_df


# '스페셜', '하이라이트', '스페셜 1부' 회차가 있는 경우 제외
def remove_special_and_highlight(rating_df):
    rating_df = rating_df[(rating_df['회차'] != '스페셜')\
                            & (rating_df['회차'] != '하이라이트')
                            & (rating_df['회차'] != '스페셜 1부')]
    rating_df.reset_index(drop=True, inplace = True)
    return rating_df



def edit_rating_df(rating_df):

    if not rating_df[rating_df['회차'].str.contains("년")].empty:
    # 연도가 두개 있는 케이스 ex) 2019년에 시작해서 2020년에 끝난 드라마
    # 연도가 바뀌는 지점을 기준으로 데이터 프레임을 두개로 나눠야 함.
        splitting_idx = list(rating_df[rating_df['회차'].str.contains("년")].index)[0]
        former_year_df = rating_df.iloc[:splitting_idx, :].copy()
        former_year_value = rating_df['방송일'][0][:5]
        latter_year_df = rating_df.iloc[splitting_idx+1:, :].copy()
        latter_year_value = rating_df['회차'][splitting_idx]

        col = "방송일"
        latter_year_df[col] = latter_year_df[col].str.replace(former_year_value,
                                                              latter_year_value)
        rating_df = pd.concat([former_year_df ,latter_year_df])

    for col in rating_df.columns:
        if col == '회차':
            rating_df[col] = rating_df[col].str.strip("제회화").astype('int32')
        elif col == "방송일":
            rating_df[col].replace('(.*)년(.*)월(.*)일',r'\1/\2/\3',
                                        regex=True, inplace = True)
            rating_df[col] = pd.to_datetime(rating_df[col])
        elif col == "시청률":
            #시청률에서 '.'대신 ','이 적힌 경우의 오류 수정 ex) 1,270->1.270
            rating_df[col] = rating_df[col].str.replace(',', '.')
            rating_df[col].replace('(.*)%(.*)', (r'\1'),
                                        regex=True, inplace = True)
            if rating_df.loc[0,'작품명'] == '우아한 친구들':
                rating_df['시청률'].iloc[-1] = rating_df['시청률'].iloc[-1].strip('%')
            rating_df[col] = rating_df[col].astype(float)

    return rating_df


## main
데이터 추출 시작

In [409]:
#데이터 추출 시작

# 시청률: AGB 시청률 중 대한민국(전국) 시청률
whole_rating_df = pd.DataFrame(columns = ['작품명', '회차', '방송일', '시청률'])

problem_list = [] # 시청률 추출이 되지 않는 드라마들을 모아 놓은 리스트

# 0~419 사이에서 시청률을 추출할 드라마의 인덱스 선택
# (0, 420으로 하면 모든 드라마 추출)
start_drama_idx = 0
end_drama_idx = 420

for title in tqdm(drama_list.loc[start_drama_idx:end_drama_idx,"작품명"]):
    #시청률 데이터 수집을 건너뛰는 드라마 리스트
    passing_list = ['회사 가기 싫어', '생일편지', '엑스가리온', '내가 가장 예뻤을 때',
                    '브람스를 좋아하세요?', '사생활', '제발 그 남자 만나지 마요',
                    '복수해라', '쌍갑포차', '영혼수선공',
                    '연애는 귀찮지만 외로운 건 싫어!', '타임즈', '대박부동산',
                    '이벤트를 확인하세요', '비밀의 여자', '진짜가 나타났다!',
                    '금이야 옥이야', '하늘의 인연', '이번 생도 잘 부탁해',
                    '넘버스 : 빌딩숲의 감시자들', '악귀', '아씨두리안',
                    '가슴이 뛴다', '기적의 형제', '이로운 사기', '행복배틀']

    # 리갈 하이(’리갈_하이_(2019년_드라마)’이 위키에서의 제목)
    if title == '리갈 하이':
        title = '리갈_하이_(2019년_드라마)'
    elif title in passing_list:
        problem_list.append(title)
        continue

    rating_df = read_rating(title) #위키피디아에서 raw 데이터 추출

    #시청률 데이터가 없는 경우 problem_list에 추가하고 다음 드라마 검색
    if rating_df.empty:
        problem_list.append(title)

    #시청률 데이터가 있는 경우
    else:
        drama_year = rating_df.columns[0][0] #드라마의 연도 추출

        rating_df.columns = edit_multi_index(rating_df.columns) # 멀티인덱스 정리


        #평균 시청률 또는 평균 제외
        rating_df = remove_avg_rating(rating_df)

        #'방송일'이 아니라 '방송일자'로 column name이 설정된 경우, '방송일'로 변경
        if '방송일자' in rating_df.columns:
            rating_df.rename(columns = {'방송일자': '방송일'}, inplace = True)

        if title == '철인왕후':
            rating_df.rename(columns = {'대한민국(전국)':'agb'}, inplace = True)
            drama_year = '2020년'
        elif title == '스타트업':
            rating_df.rename(columns = {'대한민국(전국)[2]':'agb'}, inplace = True)
            drama_year = '2020년'


        # '회차', '방송일', 'agb'시청률 column만 추출
        rating_df = rating_df[['회차', '방송일', 'agb']].copy()


        # AGB 시청률이 대한민국(전국)과 서울(수도권) 둘 다 있는 경우
        # 서울(수도권) 시청률을 삭제
        splitted_list = ['십시일반', '미쓰리는_알고_있다', '출사표']
        #시청률이 1부, 2부 나눠져 있는 드라마 리스트
        if title in splitted_list:
            rating_df = rating_df.iloc[:,[0,1,3]].copy()
        elif list(rating_df.columns).count('agb') == 2:
            rating_df = rating_df.iloc[:,:-1].copy()

        # '스페셜', '하이라이트', '스페셜 1부' 회차가 있는 경우 제외
        rating_df = remove_special_and_highlight(rating_df)
        # 'agb'열의 이름을 '시청률'로 변환
        rating_df.rename(columns = {'agb' : '시청률'}, inplace = True)

        # '방송일' 열의 데이터를 0000년 0월 0일 형식으로 변환
        if title == '(아는 건 별로 없지만) 가족입니다': # 이미 방송일에 연도가 붙어 있음
            pass
        else:
            rating_df['방송일'] = drama_year + " " + rating_df['방송일']
        rating_df.insert(0, '작품명', title)

        rating_df = edit_rating_df(rating_df)

        whole_rating_df = pd.concat([whole_rating_df, rating_df])




whole_rating_df.reset_index(drop=True,inplace=True)
print(whole_rating_df)
print("problem list:", problem_list)


 48%|████▊     | 200/420 [00:42<00:54,  4.02it/s]<ipython-input-408-6e77b298791f>:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_df['시청률'].iloc[-1] = rating_df['시청률'].iloc[-1].strip('%')
100%|██████████| 420/420 [01:52<00:00,  3.73it/s]

          작품명  회차        방송일     시청률
0     아름다운 세상   1 2019-04-05   2.178
1     아름다운 세상   2 2019-04-06   2.926
2     아름다운 세상   3 2019-04-12   3.053
3     아름다운 세상   4 2019-04-13   3.306
4     아름다운 세상   5 2019-04-19   3.346
...       ...  ..        ...     ...
9249   닥터 차정숙  12 2023-05-21  18.493
9250   닥터 차정숙  13 2023-05-27  14.420
9251   닥터 차정숙  14 2023-05-28  18.168
9252   닥터 차정숙  15 2023-06-03  14.676
9253   닥터 차정숙  16 2023-06-04  18.546

[9254 rows x 4 columns]
problem list: ['신비아파트 외전: 기억, 하리 2', '너를 싫어하는 방법', '각색은 이미 시작됐다', '반야', '개같다 거지같다 아름답다', '보좌관 2', '루왁인간', '로스:타임:라이프 - 더 라스트 찬스', '파고', '최고의 치킨', '빅이슈', '회사 가기 싫어', '보좌관', '레벨업', '인서울 - 내가 독립하는 유일한 방법', '힙합왕 - 나스나길', '생일편지', '에이틴 / 에이틴 시즌 2', '신비아파트 외전: 연애공식 구하리', '엑스가리온', '터치', '내가 가장 예뻤을 때', '브람스를 좋아하세요?', '카이로스', '나의 위험한 아내', '사생활', '트웬티트웬티', '제발 그 남자 만나지 마요', '라이브온', '복수해라', '본 대로 말하라', '안녕 드라큘라', '하이에나', '어서와', '바람과 구름과 비', '행복의 진수', '쌍갑포차', '외출', '영혼수선공', '야식남녀', '연애는 귀찮지만 외로운 건 싫어!', '시네마틱드라마 SF8', '좀비탐정', '악의 꽃', '괴물'

## Result
추출한 전체 시청률 데이터

In [410]:
#추출한 전체 시청률 데이터
whole_rating_df

,작품명,회차,방송일,시청률
0,아름다운 세상,1,2019-04-05,2.178
1,아름다운 세상,2,2019-04-06,2.926
2,아름다운 세상,3,2019-04-12,3.053
3,아름다운 세상,4,2019-04-13,3.306
4,아름다운 세상,5,2019-04-19,3.346
...,...,...,...,...
9249,닥터 차정숙,12,2023-05-21,18.493
9250,닥터 차정숙,13,2023-05-27,14.420
9251,닥터 차정숙,14,2023-05-28,18.168
9252,닥터 차정숙,15,2023-06-03,14.676


## problem_list
시청률 데이터 추출이 제대로 되지 않은 드라마 리스트

In [411]:
problem_list

['신비아파트 외전: 기억, 하리 2',
 '너를 싫어하는 방법',
 '각색은 이미 시작됐다',
 '반야',
 '개같다 거지같다 아름답다',
 '보좌관 2',
 '루왁인간',
 '로스:타임:라이프 - 더 라스트 찬스',
 '파고',
 '최고의 치킨',
 '빅이슈',
 '회사 가기 싫어',
 '보좌관',
 '레벨업',
 '인서울 - 내가 독립하는 유일한 방법',
 '힙합왕 - 나스나길',
 '생일편지',
 '에이틴 / 에이틴 시즌 2',
 '신비아파트 외전: 연애공식 구하리',
 '엑스가리온',
 '터치',
 '내가 가장 예뻤을 때',
 '브람스를 좋아하세요?',
 '카이로스',
 '나의 위험한 아내',
 '사생활',
 '트웬티트웬티',
 '제발 그 남자 만나지 마요',
 '라이브온',
 '복수해라',
 '본 대로 말하라',
 '안녕 드라큘라',
 '하이에나',
 '어서와',
 '바람과 구름과 비',
 '행복의 진수',
 '쌍갑포차',
 '외출',
 '영혼수선공',
 '야식남녀',
 '연애는 귀찮지만 외로운 건 싫어!',
 '시네마틱드라마 SF8',
 '좀비탐정',
 '악의 꽃',
 '괴물',
 '마우스',
 '아이를 찾습니다',
 '나빌레라',
 '타임즈',
 '오! 주인님',
 '라켓소년단',
 '다크홀',
 '어쩌다 가족',
 '대박부동산',
 '언더커버',
 '멀리서 보는 푸른 봄',
 '월간 집',
 '미치지 않고서야',
 '보쌈 - 운명을 훔치다',
 '이벤트를 확인하세요',
 '슬기로운 의사생활2',
 '바람피면 죽는다',
 '검은 태양',
 '뫼비우스 : 검은 태양',
 '유미의 세포들',
 '키마이라',
 '너의 밤이 되어줄게',
 '쇼윈도: 여왕의 집',
 '지리산',
 '아이돌: 더 쿠데타',
 '한 사람만',
 '트레이서',
 '학교 2021',
 '엉클',
 '설강화',
 '술꾼도시여자들',
 '으라차차 내 인생',
 '크레이지 러브',
 '내과 박원장',
 '내일',
 '지금부터, 쇼타임!',
 '아직 최선을 다하지 않았을 뿐',
 '구

In [413]:
problem_list_df = pd.DataFrame(problem_list, columns=['작품명'])
problem_list_df

,작품명
0,"신비아파트 외전: 기억, 하리 2"
1,너를 싫어하는 방법
2,각색은 이미 시작됐다
3,반야
4,개같다 거지같다 아름답다
...,...
114,아씨두리안
115,가슴이 뛴다
116,기적의 형제
117,이로운 사기


# 데이터 다운로드

In [414]:
whole_rating_df.to_csv('드라마_시청률_데이터.csv', index=False, encoding='utf-8-sig')
files.download('드라마_시청률_데이터.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [415]:
problem_list_df.to_csv('시청률_추출_못한_드라마_리스트.csv', index=False, encoding='utf-8-sig')
files.download('시청률_추출_못한_드라마_리스트.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>